In [1]:
from transformers import Wav2Vec2Processor, Wav2Vec2ForCTC
from datasets import load_dataset
import torch
 
# load model and processor
processor = Wav2Vec2Processor.from_pretrained("facebook/wav2vec2-lv-60-espeak-cv-ft")
model = Wav2Vec2ForCTC.from_pretrained("facebook/wav2vec2-lv-60-espeak-cv-ft")

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'Wav2Vec2PhonemeCTCTokenizer'. 
The class this function is called from is 'Wav2Vec2CTCTokenizer'.


In [3]:
def speech_file_to_array_fn(batch):
    import torchaudio
    speech_array, sampling_rate = torchaudio.load(batch["path"])
    batch["audio"] = speech_array[0].numpy()
    batch["sampling_rate"] = 16_000
    #batch["target_text"] = batch["sentence"]
    return batch

In [4]:
from pathlib import Path
_BASE = Path("/media/phonetics/asr_data_irish/audio/")

In [5]:
MSF = _BASE / "mul_mo_sceal_fein" / "wav"

In [6]:
msf_files = [str(f) for f in MSF.glob("*.wav")]

In [7]:
from datasets import Dataset
dataset = Dataset.from_dict({
    "path": msf_files,
})

In [8]:
dataset = dataset.map(speech_file_to_array_fn)

  0%|          | 0/3634 [00:00<?, ?ex/s]

In [ ]:
model.cuda()

In [ ]:
inputs = processor(dataset["audio"], sampling_rate=16_000, return_tensors="pt", padding=True)
#input_values = processor(msf_ds, return_tensors="pt").input_values

In [ ]:
# retrieve logits
with torch.no_grad():
   logits = model(input_values).logits

# take argmax and decode
predicted_ids = torch.argmax(logits, dim=-1)
transcription = processor.batch_decode(predicted_ids)